# Bewegtes Elektron im magnetischen/elektrischen Feld

## Magnetisches Feld

### Lorentzkraft

$$
\vec{F}_{L} = q \cdot \vec{v} \times \vec{B}
$$

### Rechte-Hand-Regel
- Daumen: Ladungsbewegung (Vorzeichen beachten!)
- Zeigefinger: Feldlinien
- Mittelfinger: resultierende Kraft

## Elektrisches Feld

### Coulomb-Kraft

$$
\vec{F}_{C} = q \cdot \vec{E}
$$

In [3]:
import ipywidgets as widgets
from ipywidgets import Box, HBox, VBox, Layout, Image
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from math import pi, sin, cos, atan, sqrt

colors = {
    "black" : "#000000",
    "U"     : "#3333cc",
    "I"     : "#e8082c",
    "B"     : "#00d000",
    "E"     : "#e8082c",
    "lgrey" : "#f0f0f0",
    "dgrey" : "#c0c0c0",
    "elec"  : "#ffff00"
}

In [2]:
class ElektronImFeld():
    
    
    
    def __init__(self):
        
        # Layer Setup
        layers = ["bg",
                  "vx",
                  "E",
                  "B",
                  "F",
                  "vy",
                  "vz",
                  "field",
                  "trace_old",
                  "trace",
                  "el"
                 ]
        lx = 700
        ly = 300
        
        self.canvas = MultiCanvas(len(layers), width=lx, height=ly)
        self.c = {}
        for li in range(len(layers)):
            self.c[layers[li]] = self.canvas[li]
        
        # Widget Setup
        self.v_slider = widgets.IntSlider(min   = 0,
                                          max   = 10,
                                          value = 1,
                                          description = r"$v_{x}$")
        
        self.v_slider = widgets.SelectionSlider(options = [0, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.8, 4],
                                                value = 1,
                                                description = r"$v_{x}$")
        
        self.field_slider = widgets.FloatSlider(min   = -1,
                                                max   =  1,
                                                value =  0,
                                                description = r"Feldstärke")
        self.field_slider.observe(self.draw_field, "value")
        
        self.z_slider = widgets.FloatSlider(min   = -1,
                                            max   =  1,
                                            value =  0,
                                            description = r"$z$")
        self.z_slider.observe(self.redraw, "value")
        
        self.BE_radio = widgets.RadioButtons(options=["B", "E"],
                                                     description="Feldtyp:",
                                                     value="B")
        self.BE_radio.observe(self.draw_field, "value")
        
        self.play = widgets.Play(
            value = 0,
            min = 0,
            max = 1400,
            step = 5,
            interval = 25,
            disabled = False,
            _repeat = True,
            show_repeat = False
        )
        self.play.observe(self.redraw)
        
        self.reset()
        self.t1 = 0
        self.tr_tmp = Canvas()
        
        # Darstellung
        display(self.canvas)
        display(self.v_slider,
                self.field_slider,
                self.BE_radio,
                self.play)
        
        # Routine
        self.draw_background()
        self.draw_field()
        self.redraw()
        
        
        
    def reset(self):
        
        # Startwerte für Variablen setzen
        self.x  = 25
        self.vx = self.v_slider.value
        self.y  = 125
        self.vy = 0
        self.z  = 100
        self.vz = 0

        
        
    def draw_arrow(self, layer, xy, angle, length, color="#000000", line_width=2, tip_width=10, hold=True):
        
        sin_a = sin((angle + 90) * pi / 180)
        cos_a = cos((angle + 90) * pi / 180)

        if length < 15:
            tip_length = 15 + (length - 15)
            width = tip_width / 2 * length / 15
        else:
            tip_length = 15
            width = tip_width / 2
        line_end = (xy[0] + (length - tip_length) * sin_a,
                    xy[1] + (length - tip_length) * cos_a)        
        
        def do_draw(layer):
            
            # Pfeilschaft
            layer.line_width = line_width
            layer.stroke_style = color
            layer.fill_style   = color
            layer.begin_path()
            layer.move_to(xy[0], xy[1])
            layer.line_to(line_end[0], line_end[1])
            layer.stroke()
            layer.close_path()
            
            # Pfeilspitze
            layer.move_to(xy[0] + length * sin_a,
                          xy[1] + length * cos_a)
            layer.line_to(line_end[0] + width * sin((angle + 180) * pi / 180),
                          line_end[1] + width * cos((angle + 180) * pi / 180))
            layer.line_to(line_end[0] - width * sin((angle + 180) * pi / 180),
                          line_end[1] - width * cos((angle + 180) * pi / 180))
            layer.fill()
        
        if hold:
            with hold_canvas(layer):
                do_draw(layer)
        else:
            do_draw(layer)
    
    
    
    def draw_background(self, color=colors["dgrey"]):
        
        bg = self.c["bg"]
        bg.set_line_dash([3, 3])
        bg.line_width = 1
        bg.stroke_style = color
        bg.begin_path()
        bg.move_to(123,  23)
        bg.line_to(123, 277)
        bg.line_to(577, 277)
        bg.line_to(577,  23)
        bg.line_to(123,  23)
        bg.stroke()
        
        
    
    def redraw(self, *args):
        
        self.draw_electron()
    
    
    
    def draw_electron(self):
        
        el = self.c["el"]
        tr = self.c["trace"]
        tr_tmp = self.tr_tmp
        tr_old = self.c["trace_old"]

        fs = self.field_slider.value / 50
        vx = self.v_slider.value
        
        # Zeitschritt berechnen
        t  = self.play.value
        dt = t - self.t1
        if dt < 0: dt += self.play.max + self.play.step
        self.t1 = t
        
        # Ortsschritte berechnen
        dx = vx * dt
        
        if self.BE_radio.value == "B":
            ay = 0
            az =  fs * vx / 330
            
        elif self.BE_radio.value == "E":
            ay = -fs / 250
            az = 0
                
        self.x += vx
        x = self.x

        if 125 < x < 575:
                    
            self.vy += ay * dt
            self.vz += az * dt

        self.y += self.vy * dt

        self.z += self.vz * dt

        y = self.y + 25
        z = self.z
        r = z**2 / 1000 + 2

        # Elektron verlässt Zeichengebiet
        if x > 650 or not(0 < y < 300) or not(0 < z < 200):

            self.play._playing = False
            self.reset()
            tr.clear()
            tr_old.clear()
            tr_old.draw_image(tr_tmp)
            self.tr_tmp = Canvas()
            x = 25
            dt = 0
            
        # Spuren zeichnen
        tr.begin_path()
        tr.stroke_style = colors["elec"]
        tr.line_width = 1
        with hold_canvas(tr):
            tr.stroke_arc(x, y, 1, 0, 2 * pi)
        
        tr_tmp.begin_path()
        tr_tmp.stroke_style = colors["lgrey"]
        tr_tmp.line_width = 1
        with hold_canvas(tr_tmp):
            tr_tmp.stroke_arc(x, y, 1, 0, 2 * pi)
        
        # Elektron zeichnen
        with hold_canvas(el):
            
            el.clear()
            el.fill_style = colors["elec"]
            el.fill_arc(x, y, r, 0, 2 * pi)
            el.stroke_style = colors["black"]
            el.stroke_arc(x, y, r, 0, 2 * pi)



    def draw_field(self, *args):
        
        f = self.c["field"]
        f.clear()
        
        fv = self.field_slider.value * 100
        
        if fv > 0:
            vz =   1
            y  =  25
        if fv < 0:
            vz =  -1
            y  = 275
        na = 0
        
        if fv != 0:
            
            na = 2 + 0.90 * (abs(fv) / 5)
            da = 450 / na
            x0 = 350 - (da * (na - 1)) / 2
            
            tmp = Canvas()
            
            for i in range(int(na)):
                
                with hold_canvas(tmp):
                    
                    self.draw_arrow(layer = tmp,
                                    xy = (x0 + da*i, y),
                                    angle = vz * 270,
                                    color = colors[self.BE_radio.value],
                                    length = 250,
                                    tip_width = 6,
                                    hold = False)
            
            f.clear()
            f.draw_image(tmp)



if __name__ == "__main__":
    
    EF = ElektronImFeld()

MultiCanvas(height=300)

SelectionSlider(description='$v_{x}$', index=1, options=(0, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 3…

FloatSlider(value=0.0, description='Feldstärke', max=1.0, min=-1.0)

RadioButtons(description='Feldtyp:', options=('B', 'E'), value='B')

Play(value=0, interval=25, max=1400, show_repeat=False, step=5)